# Topic keywords

Check which words are typical for topic tweets in different time frames

In [32]:
import csv
import math
import os
import pandas as pd
import re
import sys
from IPython.display import clear_output
from nltk.tokenize import TweetTokenizer
sys.path.append("/home/erikt/projects/newsgac/fasttext-runs")
import tscore

In [33]:
DATADIR = "/home/erikt/projects/puregome/data/text/"
ID = "id_str"
REPLYID = "in_reply_to_status_id_str"
TEXT = "text"
TOKEN = "token"
USER = "user"

In [34]:
def squeal(text=None):
    clear_output(wait=True)
    if not text is None: print(text)

## Count tweets with topic words

In [4]:
def countTweets(datePattern,query):
    count = 0
    fileList = sorted(os.listdir(DATADIR))
    for inFileName in fileList:
        if re.search(datePattern,inFileName):
            squeal(inFileName)
            df = pd.read_csv(DATADIR+inFileName,compression="gzip",index_col=ID)
            for i in range(0,len(df)):
                text = df.iloc[i][TEXT]
                if re.search(query,text): count += 1
    return(count)

In [5]:
def countTweetsReplies(datePattern,query):
    count = 0
    selectedIds = {}
    fileList = sorted(os.listdir(DATADIR))
    for inFileName in fileList:
        if re.search(datePattern,inFileName):
            squeal(inFileName)
            df = pd.read_csv(DATADIR+inFileName,compression="gzip",index_col=ID)
            for i in range(0,len(df)):
                text = df.iloc[i][TEXT]
                replyParent = df.iloc[i][REPLYID]
                if re.search(query,text) or replyParent in selectedIds: 
                    count += 1
                    idstr = df.index[i]
                    selectedIds[idstr] = True
    return(count)

In [ ]:
FILEPATTERN = "20200522"

for query in "corona corona|covid corona|covid|flattenthecurve corona|covid|blijfthuis corona|covid|rivm\
              corona|covid|mondkapje corona|covid|huisarts corona|covid|houvol corona|covid|zorg".split():
    count = countTweets(FILEPATTERN,query)
    print(count,query)

In [ ]:
FILEPATTERN = "20200522"

for query in "corona corona|covid corona|covid|flattenthecurve corona|covid|blijfthuis corona|covid|rivm\
              corona|covid|mondkapje corona|covid|huisarts corona|covid|houvol corona|covid|zorg".split():
    count = countTweetsReplies(FILEPATTERN,query)
    print(count,query)

**Note**: the query "zorg" produces many false positives. There are irrelevant types ("bezorgd" and "bezorgen"), irrelevant syntactical forms ("ik zorg dat" and "Zorg dat je") and even the correct sense is not always related to the pandemic topic ("zorg voor ouderen/gehandicapten").

## Find other relevant words in topic tweets

In [35]:
AT = r"@"
HASH = r"#"

def getTokensOfMatchedTweets(filePattern,query):
    fileList = sorted(os.listdir(DATADIR))
    matchTokens = {}
    nonMatchTokens = {}
    for inFileName in fileList:
        if re.search(filePattern,inFileName):
            squeal(inFileName)
            df = pd.read_csv(DATADIR+inFileName,compression="gzip",index_col=ID)
            for i in range(0,len(df)):
                text = re.sub("\\\\n"," ",df.iloc[i][TEXT])
                if re.search(query,text):
                    for token in TweetTokenizer().tokenize(text.lower()): 
                        if not token in matchTokens: matchTokens[token] = 0
                        matchTokens[token] += 1
                else:
                    for token in TweetTokenizer().tokenize(text.lower()): 
                        if not token in nonMatchTokens: nonMatchTokens[token] = 0
                        nonMatchTokens[token] += 1
    return(matchTokens,nonMatchTokens)


def readData(fileName):
    return(pd.read_csv(fileName,index_col=TOKEN).to_dict())

def writeData(data,fileName):
    pd.DataFrame(data).to_csv(fileName,index_label=TOKEN)
    
def findKeysStartingWithChar(data,char):
    keysStartingWithChar = []
    for key in data:
        try:
            if re.search(r"^"+char+r"\w",key):
                shortKey = key[1:]
                if shortKey in data: keysStartingWithChar.append(shortKey)
        except: pass
    return(keysStartingWithChar)

def combineKeysStartingWithChar(data,char,keysStartingWithChar):
    for key in keysStartingWithChar:
        data[key] += data[char+key]
        del(data[char+key])
    return(data)

def combineInitialHashAt(data):
    keysStartingWithHash = findKeysStartingWithChar(data,HASH)
    data = combineKeysStartingWithChar(data,HASH,keysStartingWithHash)
    keysStartingWithAt = findKeysStartingWithChar(data,AT)
    data = combineKeysStartingWithChar(data,AT,keysStartingWithAt)
    return(data)

In [36]:
NBROFEXAMPLES = 20

def dictTopN(dictionary,n=NBROFEXAMPLES):
    return([(x[1],x[0]) for x in dictionary.items()][0:n])

def dictBottomN(dictionary,n=NBROFEXAMPLES):
    return([(x[1],x[0]) for x in dictionary.items()][-n:])

In [37]:
NBROFTOKENS = "totalFreq"
NBROFTYPES = "nbrOfWords"
WORDFREQS = "wordFreqs"

def makeTscoreData(tokenList):
    data = { NBROFTOKENS:0, NBROFTYPES:0, WORDFREQS:{} }
    for token in tokenList:
        if not math.isnan(tokenList[token]):
            data[WORDFREQS][token] = tokenList[token]
            data[NBROFTYPES] += 1
            data[NBROFTOKENS] += tokenList[token]
    return(data)

def sortTscores(tscores):
    return({token:tscores[token] for token in sorted(tscores.keys(),key=lambda t:tscores[t],reverse=True)})

In [40]:
FILEPATTERN = "2020102[56789]|2020103" 
QUERY= r"1[.,]5[ -]*m|afstand.*hou|hou.*afstand|anderhalve[ -]*meter"
QUERY = "corona|covid"
QUERY = "reis|reizen"
QUERY = "hand.*was|was.*hand"

queryTokens = {}
nonQueryTokens = {}
tscoresDataQuery = {}
tscoresDataNonQuery = {}
queryTokens[FILEPATTERN],nonQueryTokens[FILEPATTERN] = getTokensOfMatchedTweets(FILEPATTERN,QUERY)
tscoresDataQuery[FILEPATTERN] = makeTscoreData(queryTokens[FILEPATTERN])
tscoresDataNonQuery[FILEPATTERN] = makeTscoreData(nonQueryTokens[FILEPATTERN])

20201031-23.out.gz


In [41]:
for x in dictTopN(sortTscores(tscore.computeTscore(tscoresDataQuery[FILEPATTERN],tscoresDataNonQuery[FILEPATTERN])),n=100): 
    print(round(x[0],1),x[1])

72.2 was
41.8 handen
31.2 hand
31.2 wassen
27.2 zoon
24.0 verkocht
20.7 school
19.9 .
19.1 behandeling
19.0 afstand
18.3 mishandeling
17.5 werden
17.5 boekhandel
17.2 loopt
17.1 nederlanden
17.1 baken
17.1 nergens
17.1 waarschuwen
17.1 gezakt
17.0 mochten
16.9 verlichting
16.9 @kajsaollongren
16.8 verkoopt
16.6 aldus
16.5 publiek
16.5 namens
16.3 kapot
16.3 @martinbosma_pvv
16.2 toen
16.2 getuige
16.1 boeken
16.0 https://t.co/zuqtdznsnx
16.0 filmt
16.0 diep
15.9 schokkende
15.9 1ste
15.8 op
15.7 middelbare
15.7 werd
15.7 let
15.6 meid
15.6 ongelooflijk
15.4 der
15.1 beelden
15.1 mijn
15.0 aan
15.0 lief
14.8 lastig
14.7 ooit
14.6 vandaag
14.4 zwarte
14.4 handig
14.4 ver
14.2 protocol
14.1 langer
14.1 fulmineert
14.0 boven
14.0 ethisch
14.0 geactiveerd
13.9 overleggen
13.8 . ...
13.7 jonge
13.6 houden
13.5 hij
13.4 had
13.4 schande
13.2 valt
12.9 vrijheid
12.8 kiezen
12.7 geweld
12.6 er
12.5 die
12.4 verschillende
12.1 onhandig
12.1 1,5
12.1 wie
11.9 europa
11.8 moest
11.6 handhaven
11.6

In [31]:
for x in dictBottomN(sortTscores(tscore.computeTscore(tscoresDataQuery[FILEPATTERN],tscoresDataNonQuery[FILEPATTERN])),n=20): 
    print(round(x[0],1),x[1])

-86.9 islam
-87.2 0.0
-88.0 fijne
-90.7 goedemorgen
-91.9 gogh
-96.8 moslims
-97.7 !
-99.8 😂
-102.6 luchtdruk
-103.0 km
-104.3 erdogan
-104.6 celine
-106.9 profeet
-107.3 vermist
-108.2 😘
-112.7 hpa
-118.3 mm
-131.0 ¡
-133.7 °
-142.4 ik


In [ ]:
FILENAME202002 = "query-tokens-202002-202003.csv"
FILENAME202004 = "query-tokens-202004-202005.csv"
NON = "non-"
WEEKS = {"2020020[2-8]":"20200202","20200209|2020021[0-5]":"20200209","2020021[6-9]|2020022[0-2]":"20200216","2020022[3-9]":"20200223",\
         "2020030[1-7]":"20200301","2020030[89]|2020031[0-4]":"20200308","2020031[5-9]|2020032[01]":"20200315","2020032[2-8]":"20200322",\
                                   "20200329|20200330|2020040[1-4]":"20200329",\
         "2020040[5-9]|2020041[01]":"20200405","2020041[2-8]":"20200412","20200419|2020042[0-5]":"20200419","2020042[6-9]|2020043|2020050[1-2]":"20200426",\
         "2020050[3-9]":"20200503","2020051[0-6]":"20200510","2020051[7-9]|2020052[0-3]":"20200517"}

In [ ]:
queryTokens202002 = readData(FILENAME202004)
nonQueryTokens202002 = readData(NON+FILENAME202004)
tscoresDataQuery = {}
tscoresDataNonQuery = {}
tscores = {}
for query in queryTokens202002:
    tscoresDataQuery[query] = makeTscoreData(combineInitialHashAt(queryTokens202002[query]))
    tscoresDataNonQuery[query] = makeTscoreData(combineInitialHashAt(nonQueryTokens202002[query]))
    print(query)
    tscores[query] = sortTscores(tscore.computeTscore(tscoresDataQuery[query],tscoresDataNonQuery[query]))
    for x in dictTopN(tscores[query]): print(round(x[0],1),x[1])

In [ ]:
# writeData(tscores,"tscores-202004-202005.csv")

In [ ]:
tscores202002 = readData("tscores-202002-202003.csv")
tscores202004 = readData("tscores-202004-202005.csv")

In [ ]:
TOPICS = "maatregelen mondkapje anderhalve besmet rivm ".split()

data = {}
for topic in TOPICS:
    for date in tscores202002.keys():
        if not topic in data: data[topic] = {}
        data[topic][date] = tscores202002[date][topic]
    for date in tscores202004.keys():
        data[topic][date] = tscores202004[date][topic]

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

DATEFORMAT = "%Y%m%d"

fig,ax = plt.subplots(figsize=(8,4))
ax.xaxis.set_major_formatter(DateFormatter("%d %b"))
for topic in TOPICS:
    plt.plot_date([datetime.strptime(WEEKS[date],DATEFORMAT) for date in data[topic]],list(data[topic].values()),label=topic,fmt="-")
ax.set(xlabel="date (weeks)",ylabel="t-scores")
plt.title("t-scores for relevant words, comparing selected topic tweets with unselected tweets")
plt.legend()
plt.show()

**Note**: We expected only positive scores for topic words but the graph shows that *anderhalve* had negative scores  in two weeks. This means that most of the tweets containing this word are outside our topic tweets. We checked a sample of these missing tweets and most of them turned out to be on-topic, but they lacked the words *corona* and *covid*.

In [ ]:
data["anderhalve"]

In [ ]:
for date in tscores202002: 
    print("#####",date)
    for x in dictTopN(sortTscores(tscores202002[date]),n=50): print(round(x[0],1),x[1])
for date in tscores202004:
    print("#####",date)
    for x in dictTopN(sortTscores(tscores202004[date]),n=50): print(round(x[0],1),x[1])

In [ ]:
queryTokens202004 = readData(FILENAME202004)
nonQueryTokens202004 = readData(NON+FILENAME202004)
for query in queryTokens202004:
    tscoresDataQuery[query] = makeTscoreData(combineInitialHashAt(queryTokens202004[query]))
    tscoresDataNonQuery[query] = makeTscoreData(combineInitialHashAt(nonQueryTokens202004[query]))
    print(query)
    for x in dictTopN(sortTscores(tscore.computeTscore(tscoresDataQuery[query],tscoresDataNonQuery[query]))): print(round(x[0],1),x[1])

In [ ]:
#writeData(queryTokens,"query-tokens-202004-202005.csv")
#writeData(nonQueryTokens,"non-query-tokens-202004-202005.csv")

In [ ]:
query = "2020051[7-9]|2020052[0-3]"
for x in dictTopN(sortTscores(tscore.computeTscore(tscoresDataQuery[query],tscoresDataNonQuery[query])),n=40): print(round(x[0],1),x[1])

In [ ]:
FILEPATTERN = "20200601" 
QUERY= r"1.5m|1,5m|afstand.*hou|hou.*afstand|anderhalve"

queryTokens = {}
nonQueryTokens = {}
tscoresDataQuery = {}
tscoresDataNonQuery = {}
queryTokens[FILEPATTERN],nonQueryTokens[FILEPATTERN] = getTokensOfMatchedTweets(FILEPATTERN,QUERY)
tscoresDataQuery[FILEPATTERN] = makeTscoreData(queryTokens[FILEPATTERN])
tscoresDataNonQuery[FILEPATTERN] = makeTscoreData(nonQueryTokens[FILEPATTERN])
for x in dictTopN(sortTscores(tscore.computeTscore(tscoresDataQuery[FILEPATTERN],tscoresDataNonQuery[FILEPATTERN])),n=40): 
    print(round(x[0],1),x[1])

**Note**: the query "zorg" produces many false positives. There are irrelevant types ("bezorgd" and "bezorgen"), irrelevant syntactical forms ("ik zorg dat" and "Zorg dat je") and even the correct sense is not always related to the pandemic topic ("zorg voor ouderen/gehandicapten").

## Estimate coverage of crawler by looking back for messages with replies

In [ ]:
DATEPATTERN = "20200522"

fileList = sorted(os.listdir(DATADIR))
seenIds = {}
for inFileName in fileList:
    if re.search(DATEPATTERN,inFileName):
        df = pd.read_csv(DATADIR+inFileName,compression="gzip",dtype=str)
        known = 0
        unknown = 0
        for i in range(0,len(df)):
            idstr = df.iloc[i][ID]
            seenIds[idstr] = True
            replyParent = df.iloc[i][REPLYID]
            if str(replyParent) != "nan":
                if replyParent in seenIds: known += 1
                else: unknown += 1
        print("{0} {1:.3f}".format(inFileName,round(known/(known+unknown),3)))